In [2]:
import os
print (os.getcwd())

from scripts.example_module import example_function

# Call the function
example_function()

/Users/hannahchloenicolecanon/Desktop/GitHub/ML-classification-deployment/ML-classification-app/notebook
Example function executed successfully!
